# Démonstration du projet 📝AnamnesIA🤖  
### *Simplifier et automatiser la documentation médicale en quelques clics*

## Création d'un environnement virtuel et installation des dépendances

**Remarque :** Adaptez les commandes ci-dessous en fonction de votre système d'exploitation si vous n'utilisez pas Windows.

In [ ]:
!python -m venv venv
!"venv/Scripts/activate" && pip install -r requirements.txt

Avant de commencer, assurez-vous d'avoir :
1. Un compte AWS actif et le CLI AWS configuré avec vos identifiants :
    - Access Key ID
    - Secret Access Key

    Utilisez la commande `aws configure` pour configurer le CLI.

2. Un compte OpenAI actif et la clé API OpenAI configurée :
    - Ajoutez votre clé API OpenAI dans les variables d'environnement sous le nom `OPENAI_API_KEY`. Exemple : `export OPENAI_API_KEY="votre_cle_api_openai"`

## Exemple d'Utilisation
### Importation des modules nécessaires

In [1]:
from src.utils import record_audio, upload_to_s3, start_transcription, get_transcription_uri, get_transcription, format_transcription, generate_anamnesis_data
import time

### Enregistrement de la conversation

La fonction `record_audio()` permet d'enregistrer une conversation jusqu'à ce qu'elle soit interrompue. Une fois l'enregistrement terminé, le fichier audio sera sauvegardé à l'emplacement spécifié dans le path retourné.

In [2]:
audio_path = record_audio()

Recording... Press 'q' to stop.
Recording saved as audio_20250405_142459.wav


### Téléchargement du fichier audio sur S3

La fonction `upload_to_s3()` permet de transférer un fichier audio local vers un bucket Amazon S3. Cela est nécessaire pour que le fichier puisse être utilisé par les services AWS, comme la transcription.

#### Paramètres :
- `file_path` (str) : Chemin du fichier audio local à télécharger.
- `bucket_name` (str) : Nom du bucket S3 où le fichier sera stocké. S'il n'existe pas il sera créé.

#### 🛠️ Au Boulot ! :

In [3]:
file_path = "data/conversation2.wav"  # Utilisation de l'audio d'exemple au lieu de celui enregistré
bucket_name = "anamnesia-audio-files-25"
s3_file_path = upload_to_s3(file_path, bucket_name)

Fichier data/conversation2.wav téléchargé dans le bucket S3 anamnesia-audio-files-25 sous audio/conversation2.wav


### Obtenir la transcription à partir du fichier S3

Cette étape utilise les fonctions `start_transcription` et `get_transcription` pour démarrer un travail de transcription sur AWS Transcribe et récupérer le résultat.

#### Étapes :
1. **Démarrer la transcription** : La fonction `start_transcription()` lance un travail de transcription sur AWS Transcribe en utilisant le fichier audio stocké dans S3.
2. **Récupérer la transcription** : La fonction `get_transcription()` attend la fin du travail de transcription et retourne les résultats.

#### Exemple de code :

In [4]:
transcription_job_name = f"transcription-job-{int(time.time())}"

# Étape 1 : Démarrer la transcription
start_transcription(transcription_job_name, bucket_name, s3_file_path)
print(f"Transcription démarrée avec le nom de job : {transcription_job_name}.")

Transcription démarrée.
Transcription démarrée avec le nom de job : transcription-job-1743855904.


In [5]:
# Étape 2 : Récupérer la transcription
transcription_uri = get_transcription_uri(transcription_job_name)
transcription = get_transcription(transcription_uri)

# Étape 3 : Formater la transcription
if transcription:
    transcription_text = format_transcription(transcription)
    print("Transcription récupérée correctement.")
else:
    print("La transcription n'est pas encore disponible.")

En attente de la transcription...
En attente de la transcription...
En attente de la transcription...
En attente de la transcription...
En attente de la transcription...
Transcription terminée.
Transcription récupérée coorrectement.


Voici la transcription formattée:

In [6]:
print(transcription_text)

[Début de la transcription]
Kiné : bonjour comment puis-je vous aider aujourd'hui
Patient : Bonjour j'ai une douleur au niveau du genou droit Cela a commencé après un entraînement de foot il y a une semaine C'est surtout quand je monte les escaliers que ça fait
Kiné : mal D'accord pouvez-vous me décrire la douleur un peu plus précisément c'est plutôt aigu ou lancinante
Patient : C'est plutôt aigu localisé à l'avant du genou C'est assez douloureux surtout lorsque je fais des mouvements de flexion
Kiné : Je vois et avez-vous remarqué d'autres symptômes comme un gonflement ou des craquements
Patient : oui il y a un léger gonflement mais je ne remarque pas de craquement Cela a tendance à se calmer quand je repose mon genou Très
Kiné : bien Depuis combien de temps avez-vous ce gonflement
Patient : Cela a commencé en même temps que la douleur il y a environ une semaine
Kiné : Est-ce que des positions ou des mouvements aggravent la douleur
Patient : Oui quand je suis debout longtemps ou que j

### Transformation de la Conversation Transcrite en Texte Structuré

Dans cette étape, nous utilisons la fonction `generate_anamnesis_data(transcription_text)` pour transformer le texte brut de la transcription en un dictionnaire Python structuré au format JSON. Ce format est prêt à être intégré dans des systèmes de gestion de données médicales.

#### Utilisation d'un Template Personnalisable

La fonction utilise un template situé dans le fichier `data/template.txt`. Ce template contient la structure et les instructions nécessaires pour générer les données d'anamnèse. Vous pouvez modifier ce fichier pour adapter la sortie aux besoins spécifiques de votre projet.

In [7]:
structured_data = generate_anamnesis_data(transcription=transcription_text)
for key, value in structured_data.items():
    print(f"{key}: {value}")

Template chargé depuis data/template.txt
Motif de consultation: Douleur aiguë au genou droit après entraînement de foot, localisée à l'avant du genou
Signes et symptômes: Douleur intense lors des mouvements de flexion, gonflement léger apparu il y a une semaine, exacerbation de la douleur en position debout prolongée ou durant l'activité sportive
Mode de vie: Actif, pratique régulière du football et salle de sport, travail sédentaire en tant que développeur informatique
Antécédents médicaux: Ancienne entorse à la cheville il y a quelques années


🎉 Voilà ! Les données structurées sont prêtes 🚀
Les informations issues de la transcription ont été transformées en un format structuré, prêt à être intégré dans votre solution médicale ou tout autre système de gestion de données.